# Notes


In [18]:
import duckdb
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from graphviz import Digraph
from sympy.physics.units import magnetic_density

from variables import Variables as vars

plt.style.use('../../notebook.mplstyle')

os.makedirs("./output", exist_ok=True)

In [22]:
# con.close()
con = duckdb.connect("./data/us_births.db") #, read_only=True)


In [32]:
mage_eth_c = con.execute(
    """
    SELECT b.year,
           b.mracehisp_c as ethnicity,
           CASE WHEN b.mage_c < 35 THEN '<35' ELSE '>=35' END as mage_group,
           COUNT(*) as birth_count,
           SUM(b.down_ind)::INT as ds_recorded,
           SUM(b.p_ds_lb_nt) as ds_est_no_term,
           SUM (b.ds_rec_weight) as case_weighted,
           -- SUM (b.p_ds_lb_nt * (1 - r.reduction)) as ds_est_reduction,
    FROM us_births as b
        JOIN reduction_rate_year r
            ON b.year = r.year
    GROUP BY b.year, b.mracehisp_c, mage_group
    ORDER BY b.year, b.mracehisp_c, mage_group
    """
).df()
mage_eth_c.to_csv(f"./output/case-weighted-ethnicity-age-group-{datetime.now().strftime("%Y%m%d%H%M")}.csv", index=False)
mage_eth_c

,year,ethnicity,mage_group,birth_count,ds_recorded,ds_est_no_term,case_weighted
0,1989,1,<35,2301071,1116,2157.940076,2073.5280
1,1989,1,>=35,229693,324,1356.330542,601.9920
2,1989,2,<35,576886,145,481.626133,411.8145
3,1989,2,>=35,34673,49,213.800335,139.1649
4,1989,3,<35,30718,13,26.017171,28.9276
...,...,...,...,...,...,...,...
427,2024,4,>=35,83252,37,602.030303,103.7887
428,2024,5,<35,817024,263,811.776154,729.0360
429,2024,5,>=35,174691,305,1284.921670,845.4600
430,2024,<NA>,<35,100610,32,103.949546,78.6848


In [33]:
mage_eth_c = con.execute(
    """
    SELECT b.year,
           b.mracehisp_c as ethnicity,
           b.mage_c as age,
           COUNT(*) as birth_count,
           SUM(b.down_ind)::INT as ds_recorded,
           SUM(b.p_ds_lb_nt) as ds_est_no_term,
           SUM (b.ds_rec_weight) as case_weighted,
           -- SUM (b.p_ds_lb_nt * (1 - r.reduction)) as ds_est_reduction,
    FROM us_births as b
        JOIN reduction_rate_year r
            ON b.year = r.year
    GROUP BY b.year, b.mracehisp_c, b.mage_c
    ORDER BY b.year, b.mracehisp_c, b.mage_c
    """
).df()
mage_eth_c.to_csv(f"./output/case-weighted-ethnicity-ages-{datetime.now().strftime("%Y%m%d%H%M")}.csv", index=False)
mage_eth_c

,year,ethnicity,age,birth_count,ds_recorded,ds_est_no_term,case_weighted
0,1989,1,10,22,0,0.014442,0.0000
1,1989,1,11,14,0,0.009196,0.0000
2,1989,1,12,38,0,0.024982,0.0000
3,1989,1,13,291,0,0.191523,0.0000
4,1989,1,14,2016,1,1.328818,1.8580
...,...,...,...,...,...,...,...
8487,2024,<NA>,46,155,0,4.779420,0.0000
8488,2024,<NA>,47,96,0,3.185590,0.0000
8489,2024,<NA>,48,68,1,2.389261,2.4589
8490,2024,<NA>,49,60,0,2.203487,0.0000


In [35]:
mage_eth_c = con.execute(
    """
    SELECT b.year,
           b.mracehisp_c as ethnicity,
           CASE
               WHEN b.mage_c < 20 THEN '<20'
               WHEN b.mage_c < 25 THEN '20-24'
               WHEN b.mage_c < 30 THEN '25-29'
               WHEN b.mage_c < 35 THEN '30-34'
               WHEN b.mage_c < 40 THEN '35-39'
               WHEN b.mage_c < 45 THEN '40-44'
               ELSE '>=45'
           END as mage_group,
           COUNT(*) as birth_count,
           SUM(b.down_ind)::INT as ds_recorded,
           SUM(b.p_ds_lb_nt) as ds_est_no_term,
           SUM (b.ds_rec_weight) as case_weighted,
           -- SUM (b.p_ds_lb_nt * (1 - r.reduction)) as ds_est_reduction,
    FROM us_births as b
        JOIN reduction_rate_year r
            ON b.year = r.year
    GROUP BY b.year, b.mracehisp_c, mage_group
    ORDER BY b.year, b.mracehisp_c, mage_group
    """
).df()
mage_eth_c.to_csv(f"./output/case-weighted-ethnicity-age-group-2-{datetime.now().strftime("%Y%m%d%H%M")}.csv", index=False)
mage_eth_c

,year,ethnicity,mage_group,birth_count,ds_recorded,ds_est_no_term,case_weighted
0,1989,1,20-24,620099,238,433.370522,442.2040
1,1989,1,25-29,848825,392,709.042853,728.3360
2,1989,1,30-34,590124,371,853.926053,689.3180
3,1989,1,35-39,200654,221,911.923759,410.6180
4,1989,1,40-44,28174,95,418.613618,176.5100
...,...,...,...,...,...,...,...
1507,2024,<NA>,30-34,35529,15,53.827068,36.8835
1508,2024,<NA>,35-39,20751,18,100.403183,44.2602
1509,2024,<NA>,40-44,5351,18,84.479782,44.2602
1510,2024,<NA>,<20,6602,0,4.410509,0.0000
